   # Pytorch Feedforward Neural Network
   This tutorial is based on https://github.com/yunjey/pytorch-tutorial

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'GPU: {torch.cuda.is_available()}')

GPU: True


In [3]:
PATH=f'/tmp/'
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## Define the Data

In [4]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root=f'{PATH}/data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=f'{PATH}/data', 
                                          train=False, 
                                          transform=transforms.ToTensor())
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


## Create a Model

In [5]:

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

## Define loss function and optimizer

In [31]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Start the training
    - call model (model(images))
    - calculate loss (criterion(outputs,labels)
    - backprop (optimizer.zero_grad(), loss.backward())
    - apply grad (optimizer.step())

In [32]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

            

Epoch [1/5], Step [100/600], Loss: 0.3391
Epoch [1/5], Step [200/600], Loss: 0.1346
Epoch [1/5], Step [300/600], Loss: 0.0475
Epoch [1/5], Step [400/600], Loss: 0.2569
Epoch [1/5], Step [500/600], Loss: 0.3255
Epoch [1/5], Step [600/600], Loss: 0.1096
Epoch [2/5], Step [100/600], Loss: 0.1987
Epoch [2/5], Step [200/600], Loss: 0.1378
Epoch [2/5], Step [300/600], Loss: 0.0783
Epoch [2/5], Step [400/600], Loss: 0.1062
Epoch [2/5], Step [500/600], Loss: 0.1261
Epoch [2/5], Step [600/600], Loss: 0.0666
Epoch [3/5], Step [100/600], Loss: 0.1251
Epoch [3/5], Step [200/600], Loss: 0.0803
Epoch [3/5], Step [300/600], Loss: 0.0820
Epoch [3/5], Step [400/600], Loss: 0.0259
Epoch [3/5], Step [500/600], Loss: 0.0845
Epoch [3/5], Step [600/600], Loss: 0.0947
Epoch [4/5], Step [100/600], Loss: 0.0742
Epoch [4/5], Step [200/600], Loss: 0.0704
Epoch [4/5], Step [300/600], Loss: 0.0773
Epoch [4/5], Step [400/600], Loss: 0.0658
Epoch [4/5], Step [500/600], Loss: 0.0091
Epoch [4/5], Step [600/600], Loss:

## Test the model

In [33]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.15 %
